<a href="https://colab.research.google.com/github/odunayo12/data-wrangling-in-r-n-py/blob/master/01_tensor_operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
# Jovian Commit Essentials
# Please retain and execute this cell without modifying the contents for `jovian.commit` to work
!pip install jovian --upgrade -q
import jovian
jovian.utils.colab.set_colab_file_id('1dIpeujnom2vQwZBLXkSeAOrrQcME1IUr')

> ### Assignment Instructions (delete this cell before submission)
> 
> The objective of this assignment is to develop a solid understanding of PyTorch tensors. In this assignment you will:
>
> 1. Pick 5 interesting functions related to PyTorch tensors by [reading the documentation](https://pytorch.org/docs/stable/torch.html), 
> 2. Edit this starter template notebook to illustrate their usage and publish your notebook to Jovian using `jovian.commit`. Make sure to add proper explanations too, not just code.
> 3. Submit the link to your published notebook on Jovian here: https://jovian.ai/learn/deep-learning-with-pytorch-zero-to-gans/assignment/assignment-1-all-about-torch-tensor .
> 4. (Optional) Write a blog post on [Medium](https://medium.com) to accompany and showcase your Jupyter notebook. [Embed cells from your notebook](https://medium.com/@aakashns/share-and-embed-jupyter-notebooks-online-with-jovian-ml-df709a03064e) wherever necessary.
> 5. (Optional) [Share your work](https://jovian.ai/forum/t/pytorch-functions-and-tensor-operations/13790) with the community and exchange feedback with other participants
>
>
> The recommended way to run this notebook is to click the "Run" button at the top of this page, and select "Run on Colab". Run `jovian.commit` regularly to save your progress.
> 
> Try to give your notebook an interesting title e.g. "All about PyTorch tensor operations", "5 PyTorch functions you didn't know you needed", "A beginner's guide to Autograd in PyToch", "Interesting ways to create PyTorch tensors", "Trigonometic functions in PyTorch", "How to use PyTorch tensors for Linear Algebra" etc.
>
> **IMPORTANT NOTE**: Make sure to submit a Jovian notebook link e.g. https://jovian.ai/aakashns/01-tensor-operations . Colab links will not be accepted.
>
> Remove this cell containing instructions before making a submission or sharing your notebook, to make it more presentable.
>



# Interesting ways to create PyTorch tensors

An short introduction about PyTorch and about the chosen functions. 

- function 1
- function 2
- function 3
- function 4
- function 5

Before we begin, let's install and import PyTorch

In [ ]:
# Uncomment and run the appropriate command for your operating system, if required

# Linux / Binder
# !pip install numpy torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Windows
# !pip install numpy torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# MacOS
# !pip install numpy torch torchvision torchaudio

In [7]:
# Import torch and other required modules
import torch

In this tutorial we seek to explore the use of `pytorch` functions in solving for a portfolio optimization problem: The Markowitz Mean-Variance Portfolio
Theory or Modern portfolio Theory. The MPT is a diversification-driven investment management technique that seeks to maximize expected returns by allocating the investment amount in such a way that a risky asset equally pays a high return.
Assumptions have it that investors are risk averse. That is they will only be willign to taken-on more risk, so long it pays a high enough returns.
Thus more formally, given $n$ assets, our paremeter consist of  $$\begin{align}
  w &= \begin{bmatrix}
           w_{1} \\
           w_{2} \\
           \vdots \\
           w_{n}
    \end{bmatrix} \in \mathbb{R}^{n \times 1}, \quad
  m &= \begin{bmatrix}
           m_{1} \\
           m_{2} \\
           \vdots \\
           w_{n}
    \end{bmatrix} \in \mathbb{R}^{n \times 1}, \quad
  \Sigma &= \begin{bmatrix}
            a_{11} & 0 & 0& 0\\
            0 & a_{22} & 0& 0\\
            \vdots  & \vdots & \ddots & \vdots\\
            0 & 0 & 0& a_{nn}
    \end{bmatrix} \in \mathbb{R}^{n \times n}, \quad
 e &= \begin{bmatrix}
           1 \\
           1 \\
           \vdots \\
           1
         \end{bmatrix} \in \mathbb{R}^{n \times 1}.
  \end{align}$$ 
Where:
  1. $w$ is the vector of portfolion weights (Say I have \\$100, weights refers to the portion of my \\$100 each asset class in my portfolio should take). Thus its only logical that it sums up to 1. That is, $\sum_{i=1}^nw_i = 1$;
  2. $\Sigma$ is the  covariance matrix for the returns on the assets in the portfolio;
  3. $m$ is a vector of expected returns.
  4. $e$ is a unit vector.
  5. Note that all matrix and vectors are in dimension $n$ of the number of assets in our portfolio. This tells us that the more the numer of assets in out porfolio, the more the computation required.

## Problem Statement.
Given the paremeter family above, we seek a porfolio weights that minimizes risk called the minimum variance porfolio. Thus, among all asset combinations we seek the one that yields minimum risk and maximizes profit given our investment sum.
This is stated as an optimization problem of the form:
$$\begin{align}
min \qquad w^{T}\Sigma w \\
s.t \qquad  w^{T} e = 1;
\end{align}$$
where $w^{T}\Sigma w$ is the variance of portfolio return. The problem above is satisfied or "solved" by:
$$\begin{align}
w= \frac{\Sigma^{-1}e}{e^{T} \Sigma^{-1}e}.
\end{align}$$

In the subsequent section we use applicable `pytorch` functions to sole a related problem.

## Function 1 -  torch.ones() & 
The `torch.ones(a,b)` function returns a tensor `X` of ones with dimension `a` by `b`. But if only a is supplied it returns an `1` by `4` vector of ones. the therefore create our unit vector $e$ using the function

In [46]:
# Example 1 - working (change this)
e = torch.ones(3)
print(e)

tensor([1., 1., 1.])


Closing comments about when to use this function

Let's save our work using Jovian before continuing.

In [48]:
!pip install jovian --upgrade --quiet

     |████████████████████████████████| 71kB 3.3MB/s 


In [49]:
import jovian

In [52]:
jovian.commit(project='01-tensor-operations')

[jovian] Detected Colab notebook...
[jovian] Please enter your API key ( from https://jovian.ai/ ):
API KEY: ··········
[jovian] Uploading colab notebook to Jovian...
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ai/odunayo12/01-tensor-operations


'https://jovian.ai/odunayo12/01-tensor-operations'

## Function 2 - `torch.diagonal()` and `torch.rand_like()`
For our covariance matrix $\Sigma$, which happens to be a diagonal matrix; the `torch.diagonal(a,b,...)` function comes in handy. The function, if `a,b,...` is a vector, generates a square matrix with `a,b,...` as the diagonal entries.

`torch.rand_like()`, on the other hand, is an extension of `torch.rand()`. Whereas the `torch.rand(a,b)` function generates a tensor of the random numbers from 0 upto but not including 1; `torch.rand_like()` returns random numbers that mimmick the size of tensor input. Lets see this in action

#### Example 1 - `torch.rand()`

In [8]:
y= torch.rand(3,2)
print(y)

tensor([[0.3879, 0.2070],
        [0.9035, 0.6991],
        [0.7802, 0.3239]])


#### Example 2 - `torch.rand_like()`

In [9]:
torch.rand_like(y)

tensor([[0.8450, 0.7784],
        [0.2538, 0.2844],
        [0.8739, 0.4374]])

As seen above, we need not specify dimension for the `torch.rand_like()` function. Its output rather "inherits" the dimension of its input.

In [10]:
torch.rand_like(y,1)

TypeError: ignored

Note that you cannot add increase the alter the dimension of the input vector in `torch.rand_like(y)` by specifying additional argument in it. If dimesion must be changed it has to be changed in the iput fuction. Suppose I want a 3-D tensor output, the following will do the magic.

In [11]:
y = torch.rand(3,2,1)
torch.rand_like(y)

tensor([[[0.0989],
         [0.3001]],

        [[0.7703],
         [0.3413]],

        [[0.2261],
         [0.4557]]])

#### Sigma 
In what follows, we combine `torch.rand_like()` with `torch.diag()` to generate a square matrix of dimension $e$.

In [47]:
rand_e = torch.rand_like(e)#.transpose(0,1)
#transp_rand_e =torch.transpose(rand_e)
print(rand_e)
sigma = torch.diag(rand_e)
print(sigma)

tensor([0.4586, 0.2929, 0.8981])
tensor([[0.4586, 0.0000, 0.0000],
        [0.0000, 0.2929, 0.0000],
        [0.0000, 0.0000, 0.8981]])


Closing comments about when to use this function

In [ ]:
jovian.commit(project='01-tensor-operations')

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "aakashns/01-tensor-operations" on https://jovian.ai/
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ai/aakashns/01-tensor-operations


'https://jovian.ai/aakashns/01-tensor-operations'

## Function 3 - ???

Add some explanations

In [ ]:
# Example 1 - working

Explanation about example

In [ ]:
# Example 2 - working

Explanation about example

In [ ]:
# Example 3 - breaking (to illustrate when it breaks)

Explanation about example

Closing comments about when to use this function

In [ ]:
jovian.commit(project='01-tensor-operations')

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "aakashns/01-tensor-operations" on https://jovian.ai/
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ai/aakashns/01-tensor-operations


'https://jovian.ai/aakashns/01-tensor-operations'

**bold text**## Function 4 - ???

Add some explanations

In [ ]:
# Example 1 - working

Explanation about example

In [ ]:
# Example 2 - working

Explanation about example

In [ ]:
# Example 3 - breaking (to illustrate when it breaks)

Explanation about example

Closing comments about when to use this function

In [ ]:
jovian.commit(project='01-tensor-operations')

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "aakashns/01-tensor-operations" on https://jovian.ai/
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ai/aakashns/01-tensor-operations


'https://jovian.ai/aakashns/01-tensor-operations'

## Function 5 - ???

Add some explanations

In [ ]:
# Example 1 - working

Explanation about example

In [ ]:
# Example 2 - working

Explanation about example

In [ ]:
# Example 3 - breaking (to illustrate when it breaks)

Explanation about example

Closing comments about when to use this function

In [ ]:
jovian.commit(project='01-tensor-operations')

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "aakashns/01-tensor-operations" on https://jovian.ai/
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ai/aakashns/01-tensor-operations


'https://jovian.ai/aakashns/01-tensor-operations'

## Conclusion

Summarize what was covered in this notebook, and where to go next

## Reference Links
Provide links to your references and other interesting articles about tensors
* Official documentation for tensor operations: https://pytorch.org/docs/stable/torch.html
* ...

In [ ]:
jovian.commit(project='01-tensor-operations')

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
